In [18]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_random_exponential
import os

import openai

from utils.embedding_utils import get_embedding, sliding_window

In [19]:
victim = "MarieCurie"#"Oppie"

df = pd.read_csv(f"../data/{victim}/db/df_text.csv")
df.head()

,source_type,text
0,paper,Marie Curie \n \nRayons émis par les composés...
1,paper,Marie Curie\nCurie c. 1920\nBorn Maria Salomea...
2,paper,Radium and RadioactivityBy Mme. Sklodowska Cur...
3,paper,"MOULD\n74\nCURRENT ONCOLOGY—VOLUME 14, NUMBER ..."
4,paper,"JULY,1931 REVIER'SOFMODER1V' I'HYSICS VOLUME 3..."


In [20]:
df_filtered = df

In [21]:
window_size = 256  # Length of text chunks
stride = 192 # Stride of sliding window; have a bit of overlap

text_chunks = []
embeddings = []

for i in tqdm(range(len(df_filtered))):
    text = df_filtered['text'].values[i].replace('\n', ' ').strip()
    text_chunks_i = list(sliding_window(text, window_size, stride))
    if len(text_chunks_i) > 80:  # If text too long, truncate
        text_chunks_i = text_chunks_i[:80]
    embeddings_i = [get_embedding(text) for text in text_chunks_i]
    text_chunks += text_chunks_i
    embeddings += embeddings_i

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:00<00:00,  3.61s/it]


In [22]:
embeddings = np.array(embeddings, dtype=np.float64)

In [23]:
data = [text_chunks]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['text_chunks']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

In [24]:
df.to_csv(f'../data/{victim}/db/text_chunks.csv', index=False)
np.save(f'../data/{victim}/db/embeddings.npy', embeddings)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from utils.embedding_utils import get_embedding

def semantic_search(query_embedding, embeddings):
    """Load context prompt."""
    similarities = cosine_similarity([query_embedding], embeddings)[0]
    ranked_indices = np.argsort(-similarities)
    return ranked_indices

In [32]:
emb = get_embedding("Tell me about science")#("Tell me about the pion")
semantic_search(emb, embeddings)

array([ 80, 684, 706, 528, 719, 701, 596, 589, 566, 532, 362, 573, 213,
       689, 530, 379, 561, 565, 711, 592, 753, 692, 698, 366, 569, 702,
       600,  49,  51, 496, 529, 754, 572, 712, 364, 725, 591, 703, 584,
       699, 720, 494, 536, 688, 233, 531, 503, 603, 605, 713, 376, 137,
       375, 578, 365, 533, 266,   9, 571, 497, 194,  87, 585, 724, 554,
       740, 710, 728, 374, 193, 560, 500, 558, 570, 493, 382, 586, 691,
       448, 718, 483, 525, 567, 244, 253, 730, 727, 722, 349, 583, 274,
       604, 721, 715, 693, 553, 198, 542,  90, 723, 555, 676, 171,  61,
       215,  79, 208, 611, 577,  39, 579,  50, 472, 582, 281, 219, 580,
        52, 170, 211, 726, 732, 145, 714, 527, 383,  11, 243, 750, 147,
       210, 128, 355,  63, 214, 677, 350, 562, 269, 690,  12, 129, 236,
       524, 581, 745,  10, 367, 309,  32, 136, 212,  15, 535,  66, 286,
       697,  54, 306, 127, 746, 300,   4, 334, 522, 380,  62, 149, 700,
        57, 595, 308, 207, 473, 232, 259, 246, 360, 438,  60, 56

In [33]:
text_chunks[semantic_search(emb, embeddings)[0]]

'science, as it has so often done in the past, and to facilitate and expedite by generous gifts the success of researches the inﬂuence of which may be far-reaching.'